In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import inchi
from tqdm import tqdm
from time import sleep
from tqdm.notebook import tqdm

from chembl_structure_pipeline import standardizer as ChEMBL_standardizer
from papyrus_structure_pipeline import standardize

In [6]:
#Read the raw efflux data (S: substrate, NS: non-substrate)
efflux_chembl =  pd.read_csv('raw_data/efflux/efflux_ChEMBL_query.csv', delimiter = ',')

BCRP1_S = Chem.SDMolSupplier('raw_data/efflux/BCRP1_efflux_metrabase_S.sdf')
MDR1_S = Chem.SDMolSupplier('raw_data/efflux/MDR1_efflux_metrabase_S.sdf')
MRP1_S = Chem.SDMolSupplier('raw_data/efflux/MRP1_efflux_metrabase_S.sdf')
MRP2_S = Chem.SDMolSupplier('raw_data/efflux/MRP2_efflux_metrabase_S.sdf')
MRP3_S = Chem.SDMolSupplier('raw_data//efflux/MRP3_efflux_metrabase_S.sdf')
MRP4_S = Chem.SDMolSupplier('raw_data/efflux/MRP4_efflux_metrabase_S.sdf')

BCRP1_NS = Chem.SDMolSupplier('raw_data/efflux/BCRP1_efflux_metrabase_NS.sdf')
MDR1_NS = Chem.SDMolSupplier('raw_data/efflux/MDR1_efflux_metrabase_nonS.sdf')
MRP1_NS = Chem.SDMolSupplier('raw_data/efflux/MRP1_efflux_metrabase_nonS.sdf')
MRP2_NS = Chem.SDMolSupplier('raw_data/efflux/MRP2_efflux_metrabase_nonS.sdf')
MRP3_NS = Chem.SDMolSupplier('raw_data/efflux//MRP3_efflux_metrabase_nonS.sdf')
MRP4_NS = Chem.SDMolSupplier('raw_data/efflux/MRP4_efflux_metrabase_nonS.sdf')

In [7]:
#Create target lists for data and UniProt Accession
efflux_targets_S_data = [BCRP1_S, MDR1_S, MRP1_S, MRP2_S, MRP3_S, MRP4_S]
efflux_targets_NS_data = [BCRP1_NS, MDR1_NS, MRP1_NS, MRP2_NS, MRP3_NS, MRP4_NS]
efflux_targets_UniProt= ['Q9UNQ0','P08183','P33527','Q92887','O15438','O15439']

In [4]:
#Classify compounds based on pchembl value (if >5 then substrate) and standard_Value (if > 10 000 nM)

efflux_chembl['status_efflux'] = efflux_chembl['pchembl_value'].apply(lambda pchembl: 'Substrate' if pchembl > 5 else 'Non-substrate')

efflux_chembl.head(10)

,molregno,pchembl_value,canonical_smiles,standard_inchi_key,accession,status_efflux
0,256064,4.43,COc1ccc(CCN(C)CCn2c(-c3ccc(N(C)C)cc3)nc3ccccc3...,IERNEONMXXBPJS-UHFFFAOYSA-N,P33527,Non-substrate
1,256036,6.30,COc1cc2c(cc1OC)CN(CCn1c(-c3ccc(C(C)C)cc3)nc3cc...,BVTJLRRZLUSCOP-UHFFFAOYSA-N,P33527,Substrate
2,256034,5.98,COc1cc2c(cc1OC)CN(CCn1c(-c3ccc(N(C)C)cc3)nc3cc...,HSFNWCVLEHOTNA-UHFFFAOYSA-N,P33527,Substrate
3,256427,5.14,COc1cc(C(=O)N2CCN(CCn3c(-c4ccc(N(C)C)cc4)nc4cc...,VYGKCLUENLMXGE-UHFFFAOYSA-N,P33527,Substrate
4,256469,5.35,COc1cc2c(cc1OC)CN(CCn1c(-c3cccc(N(C)C)c3)nc3cc...,KJWQAMYUAJXZCQ-UHFFFAOYSA-N,P33527,Substrate
5,256023,5.81,COc1cc2c(cc1OC)CN(CCn1c(-c3ccc(N(C)C)cc3)nc3cc...,RHHCJOJPWGOQQN-UHFFFAOYSA-N,P33527,Substrate
6,256565,5.00,CN(C)c1ccc(-c2nc3ccccc3c(=O)n2CC2CCCCC2)cc1,SULCAFYSBWZKFE-UHFFFAOYSA-N,P33527,Non-substrate
7,255803,6.40,COc1cc2c(cc1OC)CN(CCn1c(-c3ccc(N(C)C)cc3)nc3cc...,NYPGJZWAUHDNPD-UHFFFAOYSA-N,P33527,Substrate
8,256530,5.48,CN(C)c1ccc(-c2nc3ccccc3c(=O)n2CCc2ccncc2)cc1,LYGGSNRQNCFMMA-UHFFFAOYSA-N,P33527,Substrate
9,256083,6.05,COc1cc2c(cc1OC)CN(CCn1c(-c3ccc4ccccc4c3)nc3ccc...,TYAZKPZMWXPZFC-UHFFFAOYSA-N,P33527,Substrate


In [8]:
efflux_df = efflux_chembl

#Add substrates
for i in range(len(efflux_targets_S_data)):
    try:
        for mol in efflux_targets_S_data[i]:
            if mol is not None:
                
                smiles = Chem.MolToSmiles(mol)
                inchi_str = inchi.MolToInchi(mol)
                inchi_key = inchi.InchiToInchiKey(inchi_str)

                new_row = {'Molregno': 'Unknown', 'pchembl_value': 'Unknown', 
                           'canonical_smiles': smiles, 'standard_inchi_key': inchi_key, 
                           'accession': efflux_targets_UniProt[i],'status_efflux': 'Substrate'}
                efflux_df.loc[len(efflux_df)] = new_row
            
    except Exception as e:
            print(f"An error occurred with smile and inchi creation")
            continue 

#Add non-substrates
for i in range(len(efflux_targets_NS_data)):
    try: 
        for mol in efflux_targets_NS_data[i]:
            if mol is not None:
               
                smiles = Chem.MolToSmiles(mol)
                inchi_str = inchi.MolToInchi(mol)
                inchi_key = inchi.InchiToInchiKey(inchi_str)

               
                new_row = {'Molregno': 'Unknown', 'pchembl_value': 'Unknown', 
                           'canonical_smiles': smiles, 'standard_inchi_key': inchi_key, 
                           'accession': efflux_targets_UniProt[i],'status_efflux': 'Non-substrate'}
                efflux_df.loc[len(efflux_df)] = new_row

    except Exception as e:
            print(f"An error occurred with smiles and inchikey creation")
            continue 

[19:37:16] WARNING: Omitted undefined stereo

[19:37:16] WARNING: Omitted undefined stereo

[19:37:16] WARNING: Charges were rearranged

[19:37:16] WARNING: Omitted undefined stereo

[19:37:16] Explicit valence for atom # 30 N, 4, is greater than permitted
[19:37:16] ERROR: Could not sanitize molecule ending on line 2532
[19:37:16] ERROR: Explicit valence for atom # 30 N, 4, is greater than permitted
[19:37:16] WARNING: Charges were rearranged

[19:37:16] WARNING: Charges were rearranged

[19:37:16] WARNING: Proton(s) added/removed

[19:37:16] WARNING: Omitted undefined stereo

[19:37:16] WARNING: Omitted undefined stereo

[19:37:16] WARNING: Omitted undefined stereo

[19:37:16] WARNING: Omitted undefined stereo

[19:37:16] WARNING: Omitted undefined stereo

[19:37:16] WARNING: Omitted undefined stereo

[19:37:16] WARNING: Omitted undefined stereo

[19:37:16] WARNING: Metal was disconnected; Proton(s) added/removed

[19:37:16] WARNING: Ambiguous stereo: bond(s)

[19:37:16] WARNING: Cha

In [11]:
efflux_df.to_csv("raw_data/efflux/efflux_train_merged_query_results.csv", index=True)

In [12]:
efflux_df.head()

,molregno,pchembl_value,canonical_smiles,standard_inchi_key,accession
0,256064.0,4.43,COc1ccc(CCN(C)CCn2c(-c3ccc(N(C)C)cc3)nc3ccccc3...,IERNEONMXXBPJS-UHFFFAOYSA-N,P33527
1,256036.0,6.3,COc1cc2c(cc1OC)CN(CCn1c(-c3ccc(C(C)C)cc3)nc3cc...,BVTJLRRZLUSCOP-UHFFFAOYSA-N,P33527
2,256034.0,5.98,COc1cc2c(cc1OC)CN(CCn1c(-c3ccc(N(C)C)cc3)nc3cc...,HSFNWCVLEHOTNA-UHFFFAOYSA-N,P33527
3,256427.0,5.14,COc1cc(C(=O)N2CCN(CCn3c(-c4ccc(N(C)C)cc4)nc4cc...,VYGKCLUENLMXGE-UHFFFAOYSA-N,P33527
4,256469.0,5.35,COc1cc2c(cc1OC)CN(CCn1c(-c3cccc(N(C)C)c3)nc3cc...,KJWQAMYUAJXZCQ-UHFFFAOYSA-N,P33527


In [8]:
#Keep the relevant column

df=efflux_df.rename(columns={'canonical_smiles':'SMILES_raw'})
df = df[['SMILES_raw','status_efflux']]


,SMILES_raw,status_efflux
0,COc1ccc(CCN(C)CCn2c(-c3ccc(N(C)C)cc3)nc3ccccc3...,Non-substrate
1,COc1cc2c(cc1OC)CN(CCn1c(-c3ccc(C(C)C)cc3)nc3cc...,Substrate
2,COc1cc2c(cc1OC)CN(CCn1c(-c3ccc(N(C)C)cc3)nc3cc...,Substrate
3,COc1cc(C(=O)N2CCN(CCn3c(-c4ccc(N(C)C)cc4)nc4cc...,Substrate
4,COc1cc2c(cc1OC)CN(CCn1c(-c3cccc(N(C)C)c3)nc3cc...,Substrate


In [9]:
#Remove rows without SMILES

def remove_nan_smiles(df):

    df = df[~(df['SMILES_raw'].isna())]
    df = df.reset_index(drop=True)

    return df

In [10]:
#Papyrus Standardization

def create_sd_smiles(sd_mol):
    try:
        standardized_smiles =  Chem.MolToSmiles(sd_mol)
        return standardized_smiles
    except Exception as e:
        print(f"An sd_smiles error occurred: {str(e)}")
        return None
    
#Create InChI keys from standardized molecules
def mol_to_inchi_key(sd_mol):
    if sd_mol is not None:

        inchi_str = inchi.MolToInchi(sd_mol)
        inchi_key = inchi.InchiToInchiKey(inchi_str)
    else:
        inchi_key = None   
    return inchi_key

def standardize_molecule(mol):
    standardized_mol =  standardize(mol,raise_error=False )
    return standardized_mol

#Standardize 

def standardize_workflow(df_raw):
    for i in range(0,len(df_raw)):
        smiles =df_raw.at[i,'SMILES_raw']
        mol = Chem.MolFromSmiles(smiles)
        sd_mol =  standardize_molecule(mol)
        sd_smiles = create_sd_smiles(sd_mol)
        sd_inchi_key = mol_to_inchi_key(sd_mol)
        df_raw.at[i,'papyrus_SMILES'] = sd_smiles
        df_raw.at[i,'papyrus_inchi_key'] = sd_inchi_key

    print(f'df length after standardization: {len(df_raw)}')

    return df_raw

In [11]:
#Check for missing inchi key

def missing_inchi(df_raw):
    smiles_nan = df_raw['papyrus_SMILES'].isna().sum()
    inchikey_nan =df_raw['papyrus_inchi_key'].isna().sum()
    print(f'DB length: {len(df_raw)},        SMILES nan: {smiles_nan},        inchi key nan: {inchikey_nan}')

    #Remove rows with missing inchikey
    df_valid_inchi= df_raw[((df_raw['papyrus_inchi_key'].notna()))]
    print('-----remove missing inchikey----')
    print(f'updated length: {len(df_valid_inchi)}')

    return df_valid_inchi

In [12]:
#Create connectivity inchi

def inchi_first_part(inchi):
    return inchi.split('-')[0]

def create_connectivity_inchi(df):
    df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)
    
    return df

In [13]:
#Check for duplicates

def remove_duplicates(df):

    print(f'length: {len(df)}')
    inchi_un = df['inchi_connectivity'].nunique()
    print(f'unique_inchi: {inchi_un}')

    unique_counts = df.groupby('inchi_connectivity')['status_efflux'].nunique()
    duplicates_diff_class = unique_counts[unique_counts > 1].index

    print(f'Contradicting duplicates: {len(duplicates_diff_class)}')

    #Remove duplicates
    df = df[~(df['inchi_connectivity'].isin(duplicates_diff_class))]
    print(len(df))
    print(df['inchi_connectivity'].nunique())

    df=df.drop_duplicates(subset=['inchi_connectivity'], keep="first").reset_index(drop=True)
    #df['inchi_stereo'].values_counts

    return df

In [ ]:
#Code classes

def class_code(df):
    df['status_efflux'] = df['status_efflux'].replace({'Substrate': 1, 'Non-substrate': 0})

    return df


In [14]:
df = remove_nan_smiles(df)
df_sd = standardize_workflow(df)
df_valid = missing_inchi(df_sd)
df_connectivity_inchi = create_connectivity_inchi(df_valid)
df_no_duplicates = remove_duplicates(df_connectivity_inchi)
df_class_coded = class_code(df_no_duplicates)

df_class_coded.to_csv('train_data/kadar_efflux_train.csv')


[09:23:05] WARNING: Charges were rearranged

[09:23:05] WARNING: Charges were rearranged

[09:23:06] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[09:23:07] WARNING: Charges were rearranged

[09:23:07] WARNING: Charges were rearranged

[09:23:07] WARNING: Charges were rearranged

[09:23:07] WARNING: Charges were rearranged

[09:23:07] WARNING: Charges were rearranged

[09:23:07] WARNING: Charges were rearranged

[09:23:07] WARNING: Charges were rearranged

[09:23:07] WARNING: Charges were rearranged

[09:23:07] WARNING: Charges were rearranged

[09:23:07] WARNING: Charges were rearranged

[09:23:07] WARNING: Charges were rearranged

[09:23:08] WARNING: Charges were rearranged

[09:23:08] WARNING: Charges were rearranged

[09:23:08] WARNING: Charges were rearranged

[09:23:08] WARNING: Charges were rearranged

[09:23:09] WARNING: Charges were rearranged

[09:23:09] WARNING: Charges were rearranged

[09:23:09] WARNING: Charges were rearranged

[09:23:09] WARNING: Charges were rearranged

[09:23:09] WARNING: Charges were rearranged

[09:23:09] WARNING: Charges were rearranged

[09:23:09] WARNING: Charges were rearranged

[09:23:09]

An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


[09:23:10] WARNING: Omitted undefined stereo

[09:23:10] WARNING: Omitted undefined stereo

[09:23:10] WARNING: Omitted undefined stereo

[09:23:10] WARNING: Omitted undefined stereo

[09:23:10] WARNING: Omitted undefined stereo

[09:23:10] WARNING: Omitted undefined stereo



...

[09:29:18] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[09:29:19] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
df length after standardization: 8396
DB length: 8396,        SMILES nan: 724,        inchi key nan: 724
-----remove missing inchikey----
updated length: 7672
length: 7672
unique_inchi: 4119
Contradicting duplicates: 487
5560
3632


C:\Users\Lily\AppData\Local\Temp\ipykernel_11572\2591752290.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)
